# Tutorial (7): Useful functions

There are several useful functions to accelerate algorithm developments

[(0) Prepare and run UCCSD](#(0)-Prepare-and-run-UCCSD)  
[(1) Orbital Rotation](#(1)-Orbital-Rotation)  
[(2) Easy operator-state operation and `evolve()`](#(2)-Easy-operator-state-operation-and-evolve())  
 


In [1]:
# Import necessary modules
from quket import *
from quket.utils import *
import quket.config as cf

mpi4py is not imported. no MPI.


## (0) Prepare and run UCCSD
For this tutorial, we use first run SAUCCD-VQE

In [2]:
### Create QuketData for H2O ###
Q = create(basis="sto-3g", 
           ansatz="sauccsd", 
           n_orbitals =5, 
           n_electrons=6, 
           geometry = "O             ;\
                       H 1 1         ;\
                       H 1 1 2 110   "
        )
# Run UCCGD
Q.run()

Basis set = sto-3g

*** Geometry ******************************
  O     0.0000000    0.0000000    0.0000000
  H     1.0000000    0.0000000    0.0000000
  H    -0.3420201    0.0000000    0.9396926
*******************************************

Symmetry C2v : C2v(Abelian)
E[FCI]    = -74.999082234384     (Spin = 1   Ms = 0)
E[HF]     = -74.961156000638     (Spin = 1   Ms = 0)


Overwritten attributes  contract_2e  of <class 'pyscf.fci.direct_spin1_symm.FCISolver'>


Tapering-Off Results:
List of redundant qubits:  [0, 1, 2, 4]
Qubit: 0    Tau: 1.0 [Z0 Z3 Z5 Z7 Z8]
Qubit: 1    Tau: 1.0 [Z1 Z3 Z5 Z7 Z9]
Qubit: 2    Tau: 1.0 [Z2 Z3 Z6 Z7]
Qubit: 4    Tau: 1.0 [Z4 Z5]

Symmetry-forbidden pauli operators are removed.
NBasis = 7
Entered VQE driver
Performing VQE for sauccsd
Number of VQE parameters: 10
Initial configuration: |0000111111>
Convergence criteria: ftol = 1E-09, gtol = 1E-05
Derivatives: Analytical
Circuit order: Exp[T1] Exp[T2] |0>
Initial: E[sauccsd] = -74.961156000638  <S**2> = +0.000000  rho = 1  
      1: E[sauccsd] = -74.993183437851  <S**2> = +0.000000  Grad = 2.88e-01  CPU Time =    0.14964  (0.00 / step)
      2: E[sauccsd] = -74.997131045644  <S**2> = +0.000000  Grad = 1.43e-01  CPU Time =    0.05368  (0.00 / step)
      3: E[sauccsd] = -74.998746753620  <S**2> = +0.000001  Grad = 4.26e-02  CPU Time =    0.05102  (0.00 / step)
      4: E[sauccsd] = -74.998905435225  <S**2> = +0.000001  Grad = 3.17e-02  CPU Time =    0.04931  (0.00 /

## (1) Orbital Rotation
In a different tutorial, we have seen `oo()` can perform orbital optimization.  
This function calls `orbital_rotation()`, which performs
\begin{equation}
\tilde H = e^{-\hat \kappa} \hat H e^{\hat \kappa}
\end{equation}
so $\tilde H$ replaces the original Hamiltonian $\hat H$.  
Running `orbital_rotation()` not only sets up the Fermionic Hamiltonian but also transform to the qubit Hamiltonian depending on the mapping, either Jordan-Wigner or Bravyi-Kitaev mapping.

Note that $\hat \kappa$ is a single particle operator,
\begin{equation}
\hat \kappa = \sum^{all\; orbitals}_{p>q} \kappa_{pq} (a^\dagger_p a_q - a^\dagger_q a_p)
\end{equation}
and is specified by `kappa_list`, which contains the lower-triangular part of $\kappa$ as a vector (i.e., only the $p>q$ part).  
Note that $p,q$ run over all orbitals (`n_orbitals` but not only `n_active_orbitals`).  

Here we show some example with random $\kappa$.

In [3]:
import numpy as np

# set random skew K
K = np.random.rand(5,5)
K = K - K.T

# append frozen occupied and virtual space to generate the full size matrix (we have 2 frozen core orbitals in this case)
Kfull = np.zeros((Q.n_orbitals, Q.n_orbitals))
Kfull[Q.n_frozen_orbitals+Q.n_core_orbitals:Q.n_frozen_orbitals+Q.n_core_orbitals+Q.n_active_orbitals,
     Q.n_frozen_orbitals+Q.n_core_orbitals:Q.n_frozen_orbitals+Q.n_core_orbitals+Q.n_active_orbitals] = K

# use vectorize_skew to get the vector form
from quket.linalg.linalg import vectorize_skew
kappa_list = vectorize_skew(Kfull)

In [4]:
printmat(Kfull, 'Full kappa matrix')
printmat(kappa_list, 'Vectorized skew kappa')


Full kappa matrix

               0              1              2              3              4              5              6          

    0      0.0000000      0.0000000      0.0000000      0.0000000      0.0000000      0.0000000      0.0000000  
    1      0.0000000      0.0000000      0.0000000      0.0000000      0.0000000      0.0000000      0.0000000  
    2      0.0000000      0.0000000      0.0000000     -0.0759914      0.7287597     -0.6610425      0.1190447  
    3      0.0000000      0.0000000      0.0759914      0.0000000      0.2693044     -0.5128661     -0.1826995  
    4      0.0000000      0.0000000     -0.7287597     -0.2693044      0.0000000      0.4138672     -0.2397489  
    5      0.0000000      0.0000000      0.6610425      0.5128661     -0.4138672      0.0000000      0.0785234  
    6      0.0000000      0.0000000     -0.1190447      0.1826995      0.2397489     -0.0785234      0.0000000  


Vectorized skew kappa
               0          
    0      0.0000000

### Copy the QuketData and perform `orbital_rotation()`

In [5]:
Qnew = Q.copy()
Qnew.orbital_rotation(kappa_list)

#### Check the Hamiltonians defined in Q and Qnew are different.

In [6]:
print(Q.operators.Hamiltonian - Qnew.operators.Hamiltonian)

() 0.0
((0, 1), (0, 0)) -0.0725326797678072
((0, 1), (2, 0)) -0.5855920771946355
((0, 1), (4, 0)) -0.10913652932333509
((0, 1), (6, 0)) -0.31828773819825634
((0, 1), (8, 0)) 0.5908322681767669
((1, 1), (1, 0)) -0.0725326797678072
((1, 1), (3, 0)) -0.5855920771946355
((1, 1), (5, 0)) -0.10913652932333509
((1, 1), (7, 0)) -0.31828773819825634
((1, 1), (9, 0)) 0.5908322681767669
((2, 1), (0, 0)) -0.5855920771946348
((2, 1), (2, 0)) -0.5505370307161161
((2, 1), (4, 0)) -0.07359967134887004
((2, 1), (6, 0)) 0.03789667280190351
((2, 1), (8, 0)) 0.041093008713866064
((3, 1), (1, 0)) -0.5855920771946348
((3, 1), (3, 0)) -0.5505370307161161
((3, 1), (5, 0)) -0.07359967134887004
((3, 1), (7, 0)) 0.03789667280190351
((3, 1), (9, 0)) 0.041093008713866064
((4, 1), (0, 0)) -0.10913652932333454
((4, 1), (2, 0)) -0.07359967134886998
((4, 1), (4, 0)) -0.4088064238569382
((4, 1), (6, 0)) 0.08841952455807922
((4, 1), (8, 0)) -0.7347137302395057
((5, 1), (1, 0)) -0.10913652932333454
((5, 1), (3, 0)) -0.07

### Confirm by energy
We can also confirm the result by generating a quantum state
\begin{equation}
|\tilde \Phi\rangle = e^{\hat \kappa} |\Phi\rangle
\end{equation}
and testing its energy is the same as $\langle \Phi| \tilde H |\Phi\rangle$.

In [7]:
# Use the UCCSD state obtained by VQE
state = Qnew.state.copy()
print_state(state, threshold=1e-3, name='UCCSD state')
print('<Phi| Htilde | Phi> = ', Qnew.get_E(state))

UCCSD state
     Basis             Coef
| 0000111111 > : +0.9876 +0.0000i
| 0011110011 > : -0.0490 +0.0000i
| 0011111100 > : -0.0479 +0.0000i
| 0110110110 > : +0.0538 +0.0000i
| 0110111001 > : -0.0382 +0.0000i
| 1001110110 > : -0.0382 +0.0000i
| 1001111001 > : +0.0538 +0.0000i
| 1100110011 > : -0.0473 +0.0000i
| 1100111100 > : -0.0791 +0.0000i

<Phi| Htilde | Phi> =  -74.0583901131565


`Decompose_expT1_to_Givens_rotations()` generates `pauli_list` and `theta_list` required to perform $e^{\hat \kappa}$ as a quantum circuit

In [8]:
from quket.opelib.opelib import Decompose_expT1_to_Givens_rotations
pauli_list, theta_list = Decompose_expT1_to_Givens_rotations(K)

`create_exp_state()` generates a quantum state by constructing a circuit with `pauli_list` and `theta_list`.

In [9]:
kappa_state = create_exp_state(Qnew, init_state = state, pauli_list=pauli_list, theta_list=theta_list)
print_state(kappa_state,  name='kappa_state')

kappa_state
     Basis             Coef
| 0000111111 > : +0.2980 +0.0000i
| 0001101111 > : +0.3343 +0.0000i
| 0001111011 > : -0.2210 +0.0000i
| 0001111110 > : -0.1983 +0.0000i
| 0010011111 > : -0.3343 +0.0000i
| 0010110111 > : +0.2210 +0.0000i
| 0010111101 > : +0.1983 +0.0000i
| 0011001111 > : +0.2891 +0.0000i
| 0011011011 > : -0.2133 +0.0000i
| 0011011110 > : -0.1851 +0.0000i
| 0011100111 > : +0.2133 +0.0000i
| 0011101101 > : +0.1851 +0.0000i
| 0011110110 > : +0.1316 +0.0000i
| 0011111001 > : -0.1316 +0.0000i
| 0101101011 > : -0.1445 +0.0000i
| 0110001111 > : +0.1103 +0.0000i
| 0111001011 > : -0.1183 +0.0000i
| 1001001111 > : -0.1103 +0.0000i
| 1010010111 > : -0.1445 +0.0000i
| 1011000111 > : +0.1183 +0.0000i



Evaluate the energy expectation value using the __*untransformed*__ Hamiltonian, preserved in `Q`.

In [10]:
print('<Phi e^{-k} |  H  | e^{k} Phi> = ', Q.get_E(kappa_state))

<Phi e^{-k} |  H  | e^{k} Phi> =  -74.05839011323411


You should have obtained the same energy.

Caution: Performing `initialize()` after `orbital_rotation()` will result in the original Hamiltonian computed with the Hartree-Fock state.

# (2) Easy operator-state operation and evolve()
In some cases, one may want to apply an arbitrary non-unitary operator, for example $\hat H$, to a quantum state.  
Such an operation is not trivial with quantum computers but the state-vector implementation of Quket allows us to do it.  

As described in Tutorial-0, Quket extends the usabilities of `openfermion`'s `FermionOperator` and `QubitOperator` such that they can be directly applied to `Qulacs`' `QuantumState`.  
It also adds some easy-manipulations for `QuantumState` so that it can be added or subtracted.  

### Applying H to UCCSD state
Applying an operator (or scalar value) to a qunatum state can be performed by `@` and `*`

In [11]:
H = Q.operators.qubit_Hamiltonian
psi = Q.state.copy()
E = Q.energy

Hpsi = H @ psi
Epsi = E * psi

The resulting instance is again `QuantumState`.

In [12]:
print_state(Hpsi)

     Basis             Coef
| 0000111111 > : -74.0684 +0.0000i
| 0001111011 > : +1.7937 +0.0000i
| 0010110111 > : -1.7937 +0.0000i
| 0011001111 > : +2.1270 +0.0000i
| 0011110011 > : +3.6779 +0.0000i
| 0011111100 > : +3.5892 +0.0000i
| 0101111010 > : +1.1945 +0.0000i
| 0110110110 > : -4.0335 +0.0000i
| 0110111001 > : +2.8663 +0.0000i
| 0111110010 > : -0.1093 +0.0000i
| 1001110110 > : +2.8663 +0.0000i
| 1001111001 > : -4.0335 +0.0000i
| 1010110101 > : +1.1945 +0.0000i
| 1011110001 > : +0.1093 +0.0000i
| 1100001111 > : +0.9975 +0.0000i
| 1100110011 > : +3.5462 +0.0000i
| 1100111100 > : +5.9291 +0.0000i
| 1101111000 > : -0.1525 +0.0000i
| 1110110100 > : +0.1525 +0.0000i
| 1111000011 > : -0.1512 +0.0000i
| 1111001100 > : -0.2193 +0.0000i
| 1111110000 > : -0.8176 +0.0000i



In [13]:
print_state(Epsi)

     Basis             Coef
| 0000111111 > : -74.0684 +0.0000i
| 0001111011 > : +1.7934 -0.0000i
| 0010110111 > : -1.7934 +0.0000i
| 0011001111 > : +2.1270 -0.0000i
| 0011110011 > : +3.6781 -0.0000i
| 0011111100 > : +3.5893 -0.0000i
| 0101111010 > : +1.1945 -0.0000i
| 0110110110 > : -4.0333 +0.0000i
| 0110111001 > : +2.8665 -0.0000i
| 0111110010 > : -0.1165 +0.0000i
| 1001110110 > : +2.8665 -0.0000i
| 1001111001 > : -4.0333 +0.0000i
| 1010110101 > : +1.1945 -0.0000i
| 1011110001 > : +0.1165 -0.0000i
| 1100001111 > : +0.9975 -0.0000i
| 1100110011 > : +3.5463 -0.0000i
| 1100111100 > : +5.9292 -0.0000i
| 1101111000 > : -0.1510 +0.0000i
| 1110110100 > : +0.1510 -0.0000i
| 1111000011 > : -0.1513 +0.0000i
| 1111001100 > : -0.2186 +0.0000i
| 1111110000 > : -0.8159 +0.0000i



Check how close Schrödinger equation is satisfied:

In [14]:
residual = Hpsi - Epsi
print(residual.get_squared_norm())

0.00014220100225501286


The UCCSD state `psi` is therefore very much satisfactory.  

If the mapping of `psi` is __Jordan-Wigner__, the same operation can be performed with `FermionOperator`.  
(For the use of Bravyi-Kitaev mapping and `FermionOperator`, see `evolve()`)

In [15]:
H_fermi = get_fermion_operator(Q.operators.Hamiltonian ) ### Q.operators.Hamiltonian is InteractionOperator but not FermionOperator, so transform
Hpsi_fermi = H_fermi @ psi
Hpsi_fermi -= Hpsi
print(Hpsi_fermi.get_squared_norm())

2.516352420223088e-26


In [16]:
import quket
# Example:
n = quket.bravyi_kitaev(quket.number_operator(3))
print(n)
print(type(n))

(1.5+0j) [] +
(-0.5+0j) [Z0] +
(-0.5+0j) [Z0 Z1] +
(-0.5+0j) [Z2]
<class 'quket.lib.openfermion.QubitOperator'>


### evolve()
For a better peformance, Quket is equipped with `evolve()`, which applies an operator `A` to a quantum state `psi` in the same manner as `A*psi` (this operation actually calls `evolve()`), but with some advanced options.  

 - It accepts either `FermionOperator`, `QubitOperator`, or a list of them.  
 - `parallel=True` enables MPI parallelization.
 - `mapping='bk' or 'bravyi_kitaev'` allows to use a Bravyi-Kitaev-mapped quantum state with `FermionOperator`.  

The usage can be displayed by 
```
?evolve
```

In [17]:
Hpsi_evolve = quket.evolve(H, psi)
Hpsi_evolve1 = quket.evolve(H_fermi, psi, mapping='jw')

In [18]:
# They are the same
print((Hpsi_evolve - Hpsi_evolve1).get_squared_norm())

2.516352420223088e-26


In [19]:
# In the Bravyi-Kitaev mapping...
psi_bk = transform_state_jw2bk(psi)

Q.set(mapping='bk')
Q.initialize()

Hpsi_evolve_bk = quket.evolve(Q.operators.qubit_Hamiltonian, psi_bk)

Basis set = sto-3g

*** Geometry ******************************
  O     0.0000000    0.0000000    0.0000000
  H     1.0000000    0.0000000    0.0000000
  H    -0.3420201    0.0000000    0.9396926
*******************************************

Symmetry C2v : C2v(Abelian)
E[FCI]    = -74.999082234384     (Spin = 1   Ms = 0)
E[HF]     = -74.961156000638     (Spin = 1   Ms = 0)
Existing theta_list is inconsistent :
   size of pauli_list 27 != size of theta_list 10.
Ovewrite theta_list by zero.
Tapering-Off Results:
List of redundant qubits:  [0, 1, 5, 7]
Qubit: 0    Tau: 1.0 [Z0 Z2 Z4 Z6 Z8]
Qubit: 1    Tau: 1.0 [Z1 Z9]
Qubit: 5    Tau: 1.0 [Z5]
Qubit: 7    Tau: 1.0 [Z7 Z9]

Symmetry-forbidden pauli operators are removed.


In [20]:
residual_bk = Hpsi_evolve_bk - E * psi_bk
print(residual_bk.get_squared_norm())

0.00014220100225501326


This is of course the same as the result above with the Jordan-Wigner mapping.